In [ ]:
%run 0_local_library_setup.ipynb

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import time

In [ ]:
def spotify_get_all_results(access_token \
                            , endpoint \
                            , content_type \
                            , query = {} \
                            , max_parse_level = 0 \
                            , base_obj = None):
    api_call_headers = {
        'Authorization': 'Bearer ' + access_token,
        'Content-Type': content_type
    }

    next_api_url = endpoint

    first_call = True
    retVal_list = list()

    while next_api_url is not None:        
        api_request = requests.get(next_api_url, headers = api_call_headers, params = query if first_call else {})
        api_request.raise_for_status()

        api_request_json = api_request.json()
        
        if base_obj is not None:
            #Too simple for JMESPath...
            api_request_json = api_request_json[base_obj]

        if first_call:
            num_pages = int(np.ceil(api_request_json['total'] / api_request_json['limit']))

            desc_style = {'description_width': 'initial'}
            api_request_loading_bar = widgets.IntProgress(value = 0 \
                                                            , min = 0 \
                                                            , max = num_pages \
                                                            , description = 'Loading:' \
                                                            , bar_style = 'success' \
                                                            , orientation='horizontal' \
                                                            , style = desc_style\
                                                           )

            first_call = False

            display(api_request_loading_bar)

        next_api_url = api_request_json['next']

        retVal_list.append(pd.json_normalize(api_request_json['items'], max_level = max_parse_level))

        api_request_loading_bar.value += 1

        if api_request_loading_bar.value < num_pages:
            api_request_loading_bar.description = f'Loading Page: {api_request_loading_bar.value + 1} of {num_pages}'

    api_request_loading_bar.layout.display = 'none'
    
    return pd.concat(retVal_list).reset_index(drop = True)

In [ ]:
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))

spotify_accounts_endpoint = 'https://accounts.spotify.com/'
spotify_api_endpoint = 'https://api.spotify.com/v1/'

with open('config/config.yml', 'r') as file:
    config_contents = yaml.safe_load(file)
    
config_contents_creds = config_contents['creds']

client_id = config_contents_creds['client_id']
client_secret = config_contents_creds['client_secret']

redirect_uri = config_contents['redirect_uri']

oath_token_url = f'{spotify_accounts_endpoint}authorize?client_id={client_id}&response_type=code&redirect_uri={redirect_uri}&scope=user-read-private user-read-email playlist-read-private user-follow-read user-top-read user-read-recently-played user-library-read'

driver.get(oath_token_url)

username_input = driver.find_element('id', 'login-username')
username_input.send_keys(config_contents_creds['username'])

password_input = driver.find_element('id', 'login-password')
password_input.send_keys(config_contents_creds['password'])

login_button = driver.find_element('id', 'login-button')
login_button.click()

if (driver.current_url.startswith(f'{spotify_accounts_endpoint}en/authorize?')):
    agree_button = driver.find_element('xpath', '//button[@data-testid="auth-accept"]')
    agree_button.click()
    
time.sleep(2)
    
oauth_initial_token = driver.current_url.replace(f'{redirect_uri}/?code=', '')

driver.close()
driver.quit()

In [ ]:
base64_encoding = 'ascii'
content_type_dictionary = {'Content-Type': 'application/x-www-form-urlencoded'}

get_bearer_token_headers = {
  'Authorization': 'Basic ' + \
                    base64.b64encode(f'{client_id}:{client_secret}'.encode(base64_encoding)).decode(base64_encoding)
} | content_type_dictionary

get_bearer_token_payload = {
    'grant_type': 'authorization_code',
    'code': oauth_initial_token,
    'redirect_uri': redirect_uri
}

get_bearer_token_response = requests.post(f'{spotify_accounts_endpoint}api/token' \
                                          , headers = get_bearer_token_headers \
                                          , data = get_bearer_token_payload)
get_bearer_token_response.raise_for_status()

get_bearer_token_response_json = get_bearer_token_response.json()
access_token = get_bearer_token_response_json['access_token']

In [ ]:
track_str = 'track'
track_id_str = f'{track_str}_id'

my_tracks = spotify_get_all_results(access_token \
                                    , f'{spotify_api_endpoint}me/tracks' \
                                    , 'application/x-www-form-urlencoded' \
                                    , max_parse_level = 1)

my_tracks.columns = my_tracks.columns.str.replace(f'{track_str}.', '', regex = False)
my_tracks.rename(columns = {'id': track_id_str}, inplace = True)

display(my_tracks.head())
print(my_tracks.shape)

In [ ]:
artists_str = 'artists'

track_artists_df = convert_json_col_to_dataframe_with_key(my_tracks, track_id_str, artists_str)

display(track_artists_df.head())
print(track_artists_df.shape)

In [ ]:
id_str = 'id'
name_str = 'name'
count_track_id_str = f'count_{track_id_str}'

num_tracks_per_artist = track_artists_df.groupby([id_str, name_str])[track_id_str].count() \
                                        .to_frame().sort_values(track_id_str, ascending = False) \
                                        .reset_index().rename(columns = {track_id_str: count_track_id_str})

num_tracks_per_artist.head(10).plot(kind = 'bar', x = name_str, y = count_track_id_str, rot = 45)

plt.show()
plt.clf()

In [ ]:
display(num_tracks_per_artist[num_tracks_per_artist[count_track_id_str] >= 2][[name_str, count_track_id_str]])

In [ ]:
display(num_tracks_per_artist[num_tracks_per_artist[count_track_id_str] == 1][[name_str, count_track_id_str]])

In [ ]:
my_long_term_top_tracks = spotify_get_all_results(access_token \
                                                 , f'{spotify_api_endpoint}me/top/tracks' \
                                                 , 'application/json' \
                                                 , query = {'time_range': 'long_term'})

display(my_long_term_top_tracks)

In [ ]:
#TODO top songs by artist

In [ ]:
my_followed_artists = spotify_get_all_results(access_token \
                                              , f'{spotify_api_endpoint}me/following' \
                                              , 'application/json' \
                                              , query = {'type': 'artist'} \
                                              , base_obj = 'artists')

display(my_followed_artists)

In [ ]:
my_track_artists = num_tracks_per_artist[[id_str, name_str, count_track_id_str]]

display(my_track_artists)

In [ ]:
my_left = my_track_artists
my_right = my_followed_artists

my_followed_and_liked_artists_df = pd.merge(my_left \
                                            , my_right \
                                            , on = id_str \
                                            , how = 'inner' \
                                            , suffixes = ('', '_y'))[my_left.columns]

display(my_followed_and_liked_artists_df)
print(my_followed_and_liked_artists_df.shape)

In [ ]:
my_track_artists_ids = set(my_left[id_str])
my_followed_artists_ids = set(my_right[id_str])
my_followed_and_liked_artists_ids = set(my_followed_and_liked_artists_df[id_str])

my_unfollowed_track_artists_ids = my_track_artists_ids - my_followed_and_liked_artists_ids
my_followed_and_nonliked_artists_ids = my_followed_artists_ids - my_followed_and_liked_artists_ids

#TODO handle when my_followed_and_nonliked_artists_ids is non-empty

my_unfollowed_track_artists_df = my_left[my_left[id_str].isin(my_unfollowed_track_artists_ids)]

my_top_unfollowed_artists_indices = my_unfollowed_track_artists_df[count_track_id_str] > 2

my_top_unfollowed_artists_df = my_unfollowed_track_artists_df[my_top_unfollowed_artists_indices] \
                                    .reset_index(drop = True)

display(my_top_unfollowed_artists_df)
print(my_top_unfollowed_artists_df.shape)